# Tweet sentiment extratcion
#### Training and inference model based upon roBERTa

Inspired by [this](https://www.kaggle.com/cdeotte/tensorflow-roberta-0-705) Kaggle notebook

### Load libraries
* **Pandas** and **NumPy** for computational mathematics
* **Tensorflow** for machine learning
* **Sklearn** (StratfiedKFold) for spliting the data into balanced distributions
* **transformers** (from Hunggingface) NPL library for tensorflow 2.0
* **tokenizers** (from Huggingface) implementation of modern tokenizers

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import os

##### Global variables

In [2]:
path = os.getcwd()

### Initializer tokenizer
A tokenizer is an algorithm that transform words into symbols that a neural network can understand.

**ByteLevelBPETokenizer**
BPE (Byte-Pair Econding) tokenizer has a vocabulary that consists of single letters and sets of letters. When we create a vocabulary for this tokenizer, we start with all the letters as tokens and we merge tokens whose juxtaposition is frequent on the data set. However, if we consider UTF-8 charecters, the dictionary might get too big. To optimize our tokenizer, instead of working with letters as tokens, we use bytes as tokens.
This function requires two files as arguments. ```merges``` contains all the merged tokens and ```vocab``` contains pairs (key, value), in which keys are tokens and values are numbers as input for the neural network.

For this experiment, the files used here are available in the [Huggingface website](https://huggingface.co/roberta-base/tree/main).

In [3]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab= path + '/vocab.json',
    merges = path + '/merges.txt',
    lowercase = True, #All tokens are in lower case
    add_prefix_space=True #Do not treat spaces like part of the tokens
)

#Get the ids to decode the neural network output
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974} 

train_set = pd.read_csv(path+'/train.csv').fillna('')
train_set.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


### Loading training set

We start by initializing some auxiliar variables to parse the tweets.

* **max_lenght**: the maximum size of a tokenized tweet.
* **ct**: saves the size of the training set.
* **input_ids**: saves the IDs for each token the algorithm detected.
* **attention_mask**: As tweets have different sizes, attention_mask signals the token IDs that should be read by the model.
* **start_token**: Using a one-hot notation, saves the position where the selected fragment starts.
* **end_token**: Using a one-hot notation, saves the position where the selected fragment ends.

In [5]:
max_length = 96
ct = train_set.shape[0]
input_ids = np.ones((ct, max_length), dtype='int32')
attention_mask = np.zeros((ct, max_length), dtype='int32')
start_tokens = np.zeros((ct, max_length), dtype='int32')
end_tokens = np.zeros((ct, max_length), dtype='int32')

for i in range(ct):
    
    #Find where selected text sits inside the tweet
    text1 = " "+" ".join(train_set.loc[i, 'text'].split())
    text2 = " ".join(train_set.loc[i, 'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)] = 1
    if text1[idx-1] == ' ':
        chars[idx-1] = 1
        
    #Encode the text and find the selected_text offset, as the
    #encoded vector might not have the same length as the text vector
    enc = tokenizer.encode(text1)
    offsets = []
    idx = 0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx, idx+len(w)))
        idx += len(w)
    
    #Find the ids of the selected_text in the tokenized text
    tokens = []
    for k, (a, b) in enumerate(offsets):
        s = np.sum(chars[a:b])
        if s > 0:
            tokens.append(k)
    
    #After precessing, fill the vectors
    sent_token = sentiment_id[train_set.loc[i, 'sentiment']]
    input_ids[i, :len(enc.ids)+5] = [0] + enc.ids + [2,2] + [sent_token] + [2]
    attention_mask[i, :len(enc.ids)+5] = 1
    
    if len(tokens) > 0:
        start_tokens[i, tokens[0]+1]=1
        end_tokens[i, tokens[-1]+1] = 1

### Loading testing set

We must enconde our testing set the same way we tokenized our training set. Variables here have analogous names to those of the last case.

In [10]:
test_set = pd.read_csv(path+'/test.csv').fillna('')

ct = test_set.shape[0]
input_ids_test = np.ones((ct, max_length), dtype='int32')
attention_mask_test = np.zeros((ct, max_length), dtype='int32')

#We do not need to find the selected text for the testing set for
# the algorithm will detect and it will be compared with the fragment
# on the csv file
for i in range(ct):
    
    text1 = " " + " ".join(test_set.loc[i, 'text'].split())
    enc = tokenizer.encode(text1)
    sentiment_token = sentiment_id[test_set.loc[i, 'sentiment']]
    input_ids_test[k, :len(enc.ids)+5] = [0] + enc.ids + [2,2] + [sentiment_token] + [2]
    attention_mask_test[k, :len(enc.ids)+5] = 1